In [1]:
from data import cboe_model as cboe
import pandas as pd
import numpy as np
import plotly.express as px

In [76]:
t = 'NDX'
x = cboe.get_ticker_chains(t)
x1 = x.reset_index()
x1 = x1.fillna(0)


d:\Projects\session-extrema-model\data\cboe_model.py:444: FutureWarning:

Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead



In [82]:
x1.groupby('Expiration')[['GEX', 'Vol', 'OI']].sum()

,GEX,Vol,OI
Expiration,,,
2024-02-12,358978042,7825,1690
2024-02-13,148966500,865,775
2024-02-14,149708595,436,841
2024-02-15,112571816,486,785
2024-02-16,1483761069,2997,12154
2024-02-20,117056643,356,573
2024-02-21,36039754,143,159
2024-02-22,23521011,184,153
2024-02-23,250342468,352,1373


In [83]:
cboe.get_ticker_info(t)[0]

Symbol,^NDX
Type,index
Tick,down
Current Price,17962.4043
Open,17783.166
High,17987.8965
Low,17798.8125
Close,17962.4043
Previous Close,17962.4004
Change,179.2043
Change %,0.9977


In [ ]:

x1.loc[x1['Type'] == 'Call', 'NetGEX'] = x1['GEX']
x1.loc[x1['Type'] == 'Put', 'NetGEX'] = x1['GEX'] * -1

total_gamma = x1['NetGEX'].sum()
x2 = x1.groupby('Strike')[['NetGEX']].sum()
x3 = x1.sort_values('Expiration', ascending=False).groupby('Expiration')[['NetGEX']].sum()
last_price = cboe.get_ticker_info(t)[0]
last_price = last_price.loc['Current Price'].iloc[0]
last_trade_time = last_price.loc['Last Trade Time'].iloc[0]

fig = px.bar(x2, orientation='h')
fig.add_hline(y=last_price, line_dash="dot", line_color="cyan", name='test')

# fig.update_traces(marker=dict(color=np.where(x2['GEX'] > 0, '#3399ff', '#ff5f5f')))
fig.update_traces(
    marker=dict(color=np.where(x2['NetGEX'] > 0, '#3399ff', '#ff5f5f'), 
                line=dict(color=np.where(x2['NetGEX'] > 0, '#3399ff', '#ff5f5f'))
                )
)
fig.update_layout(
    title_text=f'Ticker: {t}<br>NetGEX: {total_gamma:,.0f}',
    font_color='#ffffff',
    plot_bgcolor='black',
    paper_bgcolor='black',
    width=600,
    height=800,
    yaxis_tickformat=',',
    xaxis=dict(showgrid=True, gridcolor='#2f2f2f')
)

top_3_strikes = x2.nlargest(3, 'NetGEX')
bottom_3_strikes = x2.nsmallest(3, 'NetGEX')

for i in top_3_strikes.index:
    fig.add_annotation(
        x=top_3_strikes.loc[i, 'NetGEX'],
        y=i,
        text=f'{i:.0f}',
        showarrow=True,
        arrowhead=1,
        ax=20,
        ay=-20
    )

for i in bottom_3_strikes.index:
    fig.add_annotation(
        x=bottom_3_strikes.loc[i, 'NetGEX'],
        y=i,
        text=f'{i:.0f}',
        showarrow=True,
        arrowhead=1,
        ax=-20,
        ay=20
    )

fig.show()


TypeError: scatter() got an unexpected keyword argument 'mode'

In [16]:
x2[x2.index > last_price]['NetGEX'].argsort()[::-1]

Strike
18850.0     4
18825.0    16
18800.0    64
18775.0    28
18750.0    40
           ..
18000.0    75
17990.0    71
17980.0    74
17975.0    60
17970.0    20
Name: NetGEX, Length: 95, dtype: int64

In [139]:
x3

,GEX
Expiration,
2024-02-09,0
2024-02-12,4275959875
2024-02-13,1501952494
2024-02-14,1104818924
2024-02-15,582778897
2024-02-16,7659928969
2024-02-20,523851648
2024-02-21,522184043
2024-02-22,144023323


In [140]:
fig = px.bar(x3[0:20], orientation='h')
# fig.add_hline(y=last_price, line_dash="dot", line_color="red", annotation_text=f'Spot: {last_price:.02f}')
# fig.update_traces(marker=dict(color=np.where(x2['GEX'] > 0, '#3399ff', '#ff5f5f')))
fig.update_traces(
    marker=dict(color=np.where(x3['GEX'] > 0, '#3399ff', '#ff5f5f'), 
                line=dict(color=np.where(x3['GEX'] > 0, '#3399ff', '#ff5f5f'))
                )
)
fig.update_layout(
    title_text=f'{t} GEX Chart by Expiry',
    font_color='#ffffff',
    plot_bgcolor='black',
    paper_bgcolor='black',
    width=600,
    height=800,
)
fig.show()

In [47]:
x.filter(like='Call', axis=0).loc[:, ("GEX")] = (
    x.loc[:, ("Gamma")]
    * 100
    * x.loc[:, ("OI")]
    * (last_price * last_price)
    * 0.01
)

x.filter(like='Put', axis=0).loc[:, ("GEX")] = (
    x.loc[:, ("Gamma")]
    * -100
    * x.loc[:, ("OI")]
    * (last_price * last_price)
    * 0.01
)

In [48]:
x

DTE       Tick  Last Price  Expected Move  % Change  \
Expiration Strike  Type                                                         
2024-02-12 17070.0 Call     2  No Change        0.00           0.00    0.0000   
                   Put      2         Up        0.66           0.01  -72.7835   
           17075.0 Call     2  No Change        0.00           0.00    0.0000   
                   Put      2       Down        0.82           0.02  -66.5306   
           17080.0 Call     2  No Change        0.00           0.00    0.0000   
...                       ...        ...         ...            ...       ...   
2026-12-18 18600.0 Put   1042  No Change        0.00           0.00    0.0000   
           18700.0 Call  1042  No Change        0.00           0.00    0.0000   
                   Put   1042  No Change        0.00           0.00    0.0000   
           18800.0 Call  1042  No Change        0.00           0.00    0.0000   
                   Put   1042  No Change        0.00           0.00    0.0000   

                         Theoretical  $ to Spot  % to Spot  Breakeven  Vol  \
Expiration Strike  Type                                                      
2024-02-12 17070.0 Call       892.08      -0.90    -0.0050   17961.50    0   
                   Put          0.97    -893.85    -4.9762   17068.55    2   
           17075.0 Call       887.09      -0.90    -0.0050   17961.50    0   
                   Put          0.98    -888.85    -4.9484   17073.55    2   
           17080.0 Call       882.10      -0.90    -0.0050   17961.50    0   
...                              ...        ...        ...        ...  ...   
2026-12-18 18600.0 Put       1820.34   -1304.40    -7.2618   16658.00    0   
           18700.0 Call      3024.95    3855.60    21.4648   21818.00    0   
                   Put       1856.17   -1240.40    -6.9055   16722.00    0   
           18800.0 Call      2972.76    3903.60    21.7321   21866.00    0   
                   Put       1892.52   -1176.40    -6.5492   16786.00    0   

                         ...       Rho  Open  High   Low  Prev Close  \
Expiration Strike  Type  ...                                           
2024-02-12 17070.0 Call  ...    1.3990  0.00  0.00  0.00      891.05   
                   Put   ...   -0.0116  0.65  0.66  0.65        0.72   
           17075.0 Call  ...    1.3994  0.00  0.00  0.00      886.05   
                   Put   ...   -0.0117  0.90  0.90  0.82        0.72   
           17080.0 Call  ...    1.3996  0.00  0.00  0.00      881.05   
...                      ...       ...   ...   ...   ...         ...   
2026-12-18 18600.0 Put   ... -230.2518  0.00  0.00  0.00     1838.00   
           18700.0 Call  ...  244.2303  0.00  0.00  0.00     3024.00   
                   Put   ... -234.1622  0.00  0.00  0.00     1874.00   
           18800.0 Call  ...  242.7954  0.00  0.00  0.00     2970.00   
                   Put   ... -238.1207  0.00  0.00  0.00     1910.00   

                         Bid Size      Bid      Ask  Ask Size  \
Expiration Strike  Type                                         
2024-02-12 17070.0 Call         1   873.50   891.50         1   
                   Put         11     1.05     1.45        11   
           17075.0 Call         1   868.50   886.50         1   
                   Put         12     1.05     1.45        10   
           17080.0 Call         1   863.50   881.50         1   
...                           ...      ...      ...       ...   
2026-12-18 18600.0 Put          1  1742.00  1942.00         1   
           18700.0 Call         1  2918.00  3118.00         1   
                   Put          1  1778.00  1978.00         1   
           18800.0 Call         1  2866.00  3066.00         1   
                   Put          1  1814.00  2014.00         1   

                                   Timestamp  
Expiration Strike  Type                       
2024-02-12 17070.0 Call                 None  
                   Put   2024-02-

In [35]:
x.columns

Index(['DTE', 'Tick', 'Last Price', 'Expected Move', '% Change', 'Theoretical',
       '$ to Spot', '% to Spot', 'Breakeven', 'Vol', 'OI', 'Delta $', 'GEX',
       'IV', 'Theta', 'Delta', 'Gamma', 'Vega', 'Rho', 'Open', 'High', 'Low',
       'Prev Close', 'Bid Size', 'Bid', 'Ask', 'Ask Size', 'Timestamp'],
      dtype='object')